# 4. Batch Inference Pipeline

## 4.1. Environment Setup
Handle repository cloning, dependency installation, and set up Python path.

In [1]:
import sys
from pathlib import Path
import hopsworks
import warnings

warnings.filterwarnings("ignore", module="IPython")

def clone_repository() -> None:
    repo_dir = Path("pm25-forecast-openmeteo-aqicn")
    if repo_dir.exists():
        print(f"Repository already exists at {repo_dir.absolute()}")
        %cd pm25-forecast-openmeteo-aqicn
    else:
        print("Cloning repository...")
        !git clone https://github.com/KristinaPalmquist/pm25-forecast-openmeteo-aqicn.git
        %cd pm25-forecast-openmeteo-aqicn

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


root_dir = Path().absolute()
for folder in ("src", "airquality", "notebooks"):
    if root_dir.parts[-1:] == (folder,):
        root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir)

if root_dir not in sys.path:
    sys.path.append(root_dir)

from utils import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
project = hopsworks.login(engine="python", api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

HopsworksSettings initialized!
2025-12-17 09:48:22,794 INFO: Initializing external client
2025-12-17 09:48:22,794 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-12-17 09:48:23,527 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-12-17 09:48:26,070 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


## 4.2. Imports

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import hopsworks
import json
from utils import airquality
from scipy.spatial.distance import cdist
import matplotlib.colors as mcolors
import os

warnings.filterwarnings("ignore")

## 4.3. Hopsworks Configuration
Establish connection to Hopsworks, retrieve API keys, connect to feature store, and get air quality and weather feature groups.

In [ ]:
secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

today = datetime.today().date()
past_date = today - timedelta(days=4)

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather",
    version=1,
)

## 4.4. Sensor Location Loading 
Load sensor location metadata from Hopsworks secrets for all sensors.

In [4]:
all_secrets = secrets.get_secrets()
locations = {}
for secret in all_secrets:
    if secret.name.startswith("SENSOR_LOCATION_JSON_"):
        sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
        location_str = secrets.get_secret(secret.name).value
        if location_str:
            locations[sensor_id] = json.loads(location_str)
print(f"Retrieved locations for {len(locations)} sensors from Hopsworks Secrets Manager.")

Retrieved locations for 105 sensors from Hopsworks Secrets Manager.


## 4.5. Weather Data Loading
Fetch recent weather data from feature store and convert date formats

In [5]:
try:
    batch_weather = weather_fg.filter(weather_fg.date >= past_date).read()
except Exception:
    batch_weather = weather_fg.read()
    batch_weather = batch_weather[batch_weather["date"] >= past_date]
batch_weather["date"] = pd.to_datetime(batch_weather["date"]).dt.tz_localize(None)
print(f"Retrieved {len(batch_weather)} weather records from Hopsworks Feature Store.")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.13s) 
Retrieved 1670 weather records from Hopsworks Feature Store.


In [35]:
print(batch_weather['date'].unique())

<DatetimeArray>
['2025-12-14 00:00:00', '2025-12-13 12:00:00', '2025-12-15 12:00:00',
 '2025-12-14 12:00:00', '2025-12-16 12:00:00', '2025-12-17 12:00:00',
 '2025-12-18 12:00:00', '2025-12-18 00:00:00', '2025-12-17 00:00:00',
 '2025-12-15 00:00:00', '2025-12-16 00:00:00', '2025-12-21 00:00:00',
 '2025-12-22 00:00:00', '2025-12-19 00:00:00', '2025-12-20 00:00:00',
 '2025-12-23 00:00:00']
Length: 16, dtype: datetime64[us]


## 4.6. Air Quality Data Loading
Fetch recent air quality with error handling for missing data.

In [6]:
try:
    batch_airquality = air_quality_fg.filter(air_quality_fg.date >= past_date).read()
    batch_airquality["date"] = pd.to_datetime(batch_airquality["date"]).dt.tz_localize(None)
except Exception:
    batch_airquality = pd.DataFrame()
print(f"Retrieved {len(batch_airquality)} air quality records from Hopsworks Feature Store.")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.15s) 
Retrieved 104 air quality records from Hopsworks Feature Store.


In [36]:
print(batch_airquality['date'].unique())

<DatetimeArray>
['2025-12-17 00:00:00']
Length: 1, dtype: datetime64[us]


## 4.7. Model Retrieval
Download trained XGBoost models from Hopsworks model registry for each sensor and extract feature names.

In [7]:
mr = project.get_model_registry()

MODEL_NAME_TEMPLATE = "air_quality_xgboost_model_{sensor_id}"

# model, model_dir, features
retrieved_models = {}

for sensor_id in locations.keys():
    model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
    retrieved_model = None

    available_models = mr.get_models(name=model_name)
    if available_models:
        retrieved_model = max(available_models, key=lambda model: model.version)

    if retrieved_model is None:
        print(f"No model found for sensor {sensor_id}, skipping...")
        continue
    
    saved_model_dir = retrieved_model.download()  
    
    import xgboost as xgb
    booster = xgb.Booster()
    booster.load_model(saved_model_dir + "/model.json")
    xgb_model = XGBRegressor()
    xgb_model._Booster = booster

    retrieved_models[sensor_id] = retrieved_model, xgb_model, booster.feature_names

Downloading: 0.000%|          | 0/509422 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/147280 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/57361 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/49557 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/31297 elapsed<00:00 remaining<?

...

Downloading: 0.000%|          | 0/524632 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/143562 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/49677 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/44342 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/31549 elapsed<00:00 remaining<?

In [30]:
print(f"Retrieved {len(retrieved_models)} models.")

Retrieved 105 models.


## 4.8. Batch Prediction Loop
Merge weather and air quality data, iteratively predict PM2.5 values for forecast days, update engineered features after each prediction, and store results

In [9]:
PREDICTION_CAP_MAX = 150.0  # Maximum reasonable PM2.5 value
PREDICTION_CAP_MIN = 0.0    # Minimum reasonable PM2.5 value

In [10]:
# Merge historical data with weather data
batch_data = pd.merge(batch_weather, batch_airquality, on=["date", "sensor_id"], how="left")
batch_data = batch_data.sort_values(["sensor_id", "date"])

In [ ]:
feature_cols = [
    "pm25_rolling_3d",
    "pm25_lag_1d",
    "pm25_lag_2d",
    "pm25_lag_3d",
    "pm25_nearby_avg",
]

In [ ]:
# Create a new column for pedicted pm2.5 values, fill with NaN for now
batch_data["predicted_pm25"] = np.nan
# Create a new column for days before forecast day, fill with NaN for now
batch_data["days_before_forecast_day"] = np.nan
# For each feature name in feature_cols list, 
# create a new column for predicted feature values, fill with NaN for now
for col in feature_cols:
    batch_data[f"predicted_{col}"] = np.nan
# Select all rows where pm25 is NaN and date is today or later
# drop any NaN date values, sort the dates in ascending order, get unique dates
# forecast days will be a list of dates for which pm2.5 predictions are needed
forecast_days = (
    batch_data.loc[batch_data["pm25"].isna() & 
                   (batch_data["date"] >= today.strftime("%Y-%m-%d")), "date"]
    .dropna()
    .sort_values()
    .unique()
)

In [ ]:
# Track sensors processed to prevent duplicates and count progress
# sensors_processed = set()
# warning_count = 0
# MAX_WARNINGS = 3  # Reduced to minimize output noise
# predictions_made = 0

In [31]:
print(forecast_days)

<DatetimeArray>
['2025-12-17 00:00:00', '2025-12-17 12:00:00', '2025-12-18 00:00:00',
 '2025-12-18 12:00:00', '2025-12-19 00:00:00', '2025-12-20 00:00:00',
 '2025-12-21 00:00:00', '2025-12-22 00:00:00', '2025-12-23 00:00:00']
Length: 9, dtype: datetime64[us]


In [14]:
for target_day in forecast_days:
    # context with all sensors up to current day
    window = batch_data.loc[batch_data["date"] <= target_day].copy()
    day_rows = window[(window["date"] == target_day) & window["pm25"].isna()]

    for _, row in day_rows.iterrows():
        sensor_id = row["sensor_id"]

        _, xgb_model, model_features = retrieved_models[sensor_id]
        features = (row.reindex(model_features).to_frame().T.apply(pd.to_numeric, errors="coerce"))
        y_hat = xgb_model.predict(features)[0]

        idx = batch_data.index[(batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)][0]
        batch_data.at[idx, "pm25"] = y_hat
        batch_data.at[idx, "predicted_pm25"] = y_hat
        batch_data.at[idx, "days_before_forecast_day"] = (target_day - pd.Timestamp(today)).days + 1

    # recompute features for all sensors now that this days values exist
    temp_df = batch_data.loc[batch_data["date"] <= target_day].copy()
    temp_df = airquality.add_rolling_window_feature(
        temp_df, window_days=3, column="pm25", new_column="pm25_rolling_3d"
    )
    temp_df = airquality.add_lagged_features(temp_df, column="pm25", lags=[1, 2, 3])
    temp_df = airquality.add_nearby_sensor_feature(
        temp_df,
        locations,
        column="pm25",
        n_closest=3,
        new_column="pm25_nearby_avg",
    )

    current_rows = temp_df[temp_df["date"] == target_day]
    for _, row in current_rows.iterrows():
        sensor_id = row["sensor_id"]
        mask = (batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)
        if mask.any():
            for col in feature_cols:
                batch_data.loc[mask, f"predicted_{col}"] = row[col]

predictions = batch_data.loc[
    batch_data["predicted_pm25"].notna(),
    ["date", "sensor_id", "predicted_pm25", "days_before_forecast_day"]
    + [f"predicted_{col}" for col in feature_cols],
].reset_index(drop=True)
batch_data.loc[batch_data["date"] > pd.Timestamp(today), "pm25"] = np.nan

## 4.9. Save Predictions
Export prediction results to CSV file in models directory.

In [ ]:
# batch_data.to_csv(f"{root_dir}/models/predictions.csv", columns=batch_data.columns, index=False)

save predictions to feature store

In [ ]:
predictions_fg = fs.get_or_create_feature_group(
    name="air_quality_predictions",
    version=1,
    primary_key=["sensor_id", "datetime"],
    description="Daily PM2.5 predictions per sensor"
)

# batch_data is the DataFrame of predictions
predictions_fg.insert(batch_data)

## 4.10. Generate Forecast Plots
Create forecast visualization plots for each sensor and upload them to Hopsworks dataset storage.

In [16]:
forecast_paths = []

for sensor_id, location in locations.items():
    sensor_forecast = predictions[predictions["sensor_id"] == sensor_id].copy()

    city, street = location["city"], location["street"]
    forecast_path = f"{root_dir}/models/{sensor_id}/images/forecast.png"
    Path(forecast_path).parent.mkdir(parents=True, exist_ok=True)

    plt = airquality.plot_air_quality_forecast(
        location["city"],
        location["street"],
        sensor_forecast,
        forecast_path,
        hindcast=False,
    )
    plt.close()
    forecast_paths.append((sensor_id, forecast_path))

dataset_api = project.get_dataset_api()
today_short = today.strftime("%Y-%m-%d")
if not dataset_api.exists("Resources/airquality"):
    dataset_api.mkdir("Resources/airquality")

for sensor_id, forecast_path in forecast_paths:
    dataset_api.upload(
        forecast_path,
        f"Resources/airquality/{sensor_id}_{today_short}_forecast.png",
        overwrite=True,
    )
print(f"Forecast plots available in Hopsworks under {project.get_url()}/settings/fb/path/Resources/airquality")

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/128095/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/154549/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/345007/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/401314/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/407335/images/forecast.png: 0.0…

...

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/80773/images/forecast.png: 0.00…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/84085/images/forecast.png: 0.00…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/88876/images/forecast.png: 0.00…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/90676/images/forecast.png: 0.00…

Forecast plots available in Hopsworks under https://c.app.hopsworks.ai:443/p/1279184/settings/fb/path/Resources/airquality


## 4.11. Insert Monitoring Data
Save predictions to monitoring feature group in Hopsworks for tracking.

In [17]:
monitor_fg = fs.get_or_create_feature_group(
    name="aq_predictions",
    description="Air Quality prediction monitoring",
    version=1,
    primary_key=["sensor_id", "date", "days_before_forecast_day"],
    event_time="date",
)

monitor_fg.insert(predictions, wait=True)

Uploading Dataframe: 100.00% |██████████| Rows 834/834 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: aq_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/aq_predictions_1_offline_fg_materialization/executions
2025-12-17 10:04:23,866 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-12-17 10:04:27,108 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-17 10:04:30,359 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-17 10:06:20,599 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-12-17 10:06:23,843 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-12-17 10:06:24,269 INFO: Waiting for log aggregation to finish.
2025-12-17 10:06:24,270 INFO: Execution finished successfully.


(Job('aq_predictions_1_offline_fg_materialization', 'SPARK'), None)

## 4.12. Hindcast Analysis
Compare predicted with forecasted values (1-day prior forecast)

In [18]:
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)

air_quality_df = air_quality_fg.read()[["date", "sensor_id", "pm25"]]
air_quality_df["date"] = pd.to_datetime(air_quality_df["date"]).dt.tz_localize(None)

for sensor_id, location in locations.items():
    try:
        sensor_preds = monitoring_df[monitoring_df["sensor_id"] == sensor_id][["date", "predicted_pm25"]]
        merged = sensor_preds.merge(
            air_quality_df[air_quality_df["sensor_id"] == sensor_id][["date", "pm25"]],
            on="date",
            how="inner",
        ).sort_values("date")

        city, street = location["city"], location["street"]
        hindcast_path = f"{root_dir}/models/{sensor_id}/images/hindcast_prediction.png"
        Path(hindcast_path).parent.mkdir(parents=True, exist_ok=True)

        plt = airquality.plot_air_quality_forecast(
            city,
            street,
            merged if not merged.empty else sensor_preds.assign(pm25=np.nan),
            hindcast_path,
            hindcast=True,
        )
        plt.close()

        dataset_api.upload(
            hindcast_path,
            f"Resources/airquality/{sensor_id}_{today:%Y-%m-%d}_hindcast.png",
            overwrite=True,
        )

    except Exception as e:
        print(f"⚠️  Error processing hindcast for sensor {sensor_id}: {e}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.24s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.27s) 


Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/128095/images/hindcast_predicti…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/154549/images/hindcast_predicti…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/345007/images/hindcast_predicti…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/401314/images/hindcast_predicti…

...

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/62848/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/80773/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/84085/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/88876/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/90676/images/hindcast_predictio…

## 4.13 IDW Heatmap
IDW - Inverse Distance Weighting

### 4.13.1 IDW interpolation function

In [19]:
def idw_interpolation(points, values, grid_points, lon_mesh, power=2):
    # compute distances between grid points and known data points 
    distances = cdist(grid_points, points)
    # replace 0 with a small value to avoid division by zero
    distances = np.where(distances == 0, 1e-10, distances)
    # compute weights based on inverse distance
    weights = 1.0 / (distances ** power)
    # sum of weights for normalization
    weights_sum = np.sum(weights, axis=1)
    # compute interpolated values - weighted average of known values for each grid point
    interpolated = np.sum(weights * values, axis=1) / weights_sum
    # reshape to the match grid shape
    return interpolated.reshape(lon_mesh.shape)

In [20]:
grid_bounds = tuple(list(json.load(open(f"{root_dir}/utils/coordinates.json")).values())[:4])
# grid_bounds = map_bounds[1], map_bounds[0], map_bounds[3], map_bounds[2]  # lat_min, lat_max, lon_min, lon_max
print(grid_bounds)

(-7.602536, 50.862218, 36.738284, 69.923179)


In [22]:
def plot_pm25_idw_heatmap(
    predictions: pd.DataFrame,
    locations: dict,
    forecast_date: datetime,
    path: str,
    grid_bounds=grid_bounds,
    grid_resolution=800,
    power=2,
):

    print(grid_bounds)

    df_day = predictions[predictions["date"] == forecast_date].copy()

    sensor_ids = [sid for sid in df_day["sensor_id"].unique() if sid in locations]
    sensor_coords = np.array([[locations[sid]["longitude"], locations[sid]["latitude"]] for sid in sensor_ids])

    pm25_values = []
    for sid in sensor_ids:
        row = df_day[df_day["sensor_id"] == sid]
        if not row.empty:
            # Use observed value if available, otherwise predicted
            val = row["pm25"].iloc[0] if not pd.isna(row["pm25"].iloc[0]) else row["predicted_pm25"].iloc[0]
            pm25_values.append(val)
        else:
            pm25_values.append(np.nan)

    pm25_values = np.array(pm25_values)
    # Cap extreme values to prevent unrealistic interpolation
    pm25_values = np.clip(pm25_values, PREDICTION_CAP_MIN, PREDICTION_CAP_MAX)


    # df_day = predictions[predictions["date"] == forecast_date].copy()

    # sensor_coords = np.array([[locations[sid]["longitude"], locations[sid]["latitude"]]
    #                           for sid in df_day["sensor_id"].unique() if sid in locations])

    # pm25_column = "predicted_pm25"
    # if df_day["predicted_pm25"].isna().any():
    #     pm25_column = "pm25"

    # pm25_values = np.array([df_day[df_day["sensor_id"] == sid][pm25_column].iloc[0]
    #                         for sid in df_day["sensor_id"].unique() if sid in locations])
    
    # # Cap extreme values to prevent unrealistic interpolation
    # pm25_values = np.clip(pm25_values, PREDICTION_CAP_MIN, PREDICTION_CAP_MAX)

    min_lon, min_lat, max_lon, max_lat = grid_bounds

    lon_grid = np.linspace(min_lon, max_lon, grid_resolution)
    lat_grid = np.linspace(min_lat, max_lat, grid_resolution)
    lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
    grid_points = np.column_stack([lon_mesh.ravel(), lat_mesh.ravel()])

    idw_result = idw_interpolation(sensor_coords, pm25_values, grid_points, lon_mesh, power=power)

    default_levels = np.array([0, 12, 35, 55, 150, 250, 500])
    category_colors = [
            "#00e400", "#7de400", "#ffff00", "#ffb000",
            "#ff7e00", "#ff4000", "#ff0000", "#d00050",
            "#8f3f97", "#7e0023"
        ]
    vmin, vmax = default_levels[0], 150
    
    clipped = np.clip(idw_result, vmin, vmax)
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(
        clipped,
        extent=(min_lon, max_lon, min_lat, max_lat),
        origin="lower",
        cmap=mcolors.LinearSegmentedColormap.from_list("aqi", category_colors, N=512),
        vmin=vmin,
        vmax=vmax,
        alpha=0.5,
    )
    ax.set_aspect('auto')

    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)
    ax.axis("off")

    fig.savefig(path, dpi=300, bbox_inches="tight", pad_inches=0, transparent=True)
    plt.close(fig)

In [ ]:
# interpolation_dir = f"{root_dir}/models/interpolation"
# if not os.path.exists(interpolation_dir):
#     os.mkdir(interpolation_dir)

# # Use predictions DataFrame which contains all forecast days with PM2.5 values
# interpolation_df = predictions.copy()

In [ ]:
# import os

# Always resolve relative to the backend project root
interpolation_dir = os.path.join(os.path.dirname(__file__), "models", "interpolation")

# Ensure the directory exists
os.makedirs(interpolation_dir, exist_ok=True)

# Use predictions DataFrame which contains all forecast days with PM2.5 values
interpolation_df = predictions.copy()

In [29]:

# Add any actual PM2.5 data from today if available
today_actual = batch_data[batch_data["date"] == today_short].copy()

if not today_actual.empty:
    # Ensure both columns exist for the plotting function
    today_actual = today_actual[[col for col in ["date", "sensor_id", "pm25", "predicted_pm25"] if col in today_actual.columns]]
    interpolation_df = pd.concat([today_actual, interpolation_df], ignore_index=True)

for i, forecast_date in enumerate(sorted(interpolation_df["date"].unique())):
    forecast_date_short = forecast_date.strftime("%Y-%m-%d")
    output_png = f"{interpolation_dir}/forecast_interpolation_{i}d.png"
    print(interpolation_df.info())

    plot_pm25_idw_heatmap(
        interpolation_df,
        locations,
        forecast_date,
        output_png,
    )

    dataset_api.upload(
        output_png,
        f"Resources/airquality/interpolation_{today_short}_{forecast_date_short}.png",
        overwrite=True,
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       1149 non-null   datetime64[us]
 1   sensor_id                  1149 non-null   object        
 2   pm25                       315 non-null    float64       
 3   predicted_pm25             837 non-null    float64       
 4   days_before_forecast_day   834 non-null    float64       
 5   predicted_pm25_rolling_3d  833 non-null    float64       
 6   predicted_pm25_lag_1d      833 non-null    float64       
 7   predicted_pm25_lag_2d      728 non-null    float64       
 8   predicted_pm25_lag_3d      623 non-null    float64       
 9   predicted_pm25_nearby_avg  834 non-null    float64       
dtypes: datetime64[us](1), float64(8), object(1)
memory usage: 89.9+ KB
None
(-7.602536, 50.862218, 36.738284, 69.923179)


Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       1149 non-null   datetime64[us]
 1   sensor_id                  1149 non-null   object        
 2   pm25                       315 non-null    float64       
 3   predicted_pm25             837 non-null    float64       
 4   days_before_forecast_day   834 non-null    float64       
 5   predicted_pm25_rolling_3d  833 non-null    float64       
 6   predicted_pm25_lag_1d      833 non-null    float64       
 7   predicted_pm25_lag_2d      728 non-null    float64       
 8   predicted_pm25_lag_3d      623 non-null    float64       
 9   predicted_pm25_nearby_avg  834 non-null    float64       
dtypes: datetime64[us](1), float64(8), object(1)
memory usage: 89.9+ KB
None
(-7.602536, 50.862218, 36.738284, 69.923179)


Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       1149 non-null   datetime64[us]
 1   sensor_id                  1149 non-null   object        
 2   pm25                       315 non-null    float64       
 3   predicted_pm25             837 non-null    float64       
 4   days_before_forecast_day   834 non-null    float64       
 5   predicted_pm25_rolling_3d  833 non-null    float64       
 6   predicted_pm25_lag_1d      833 non-null    float64       
 7   predicted_pm25_lag_2d      728 non-null    float64       
 8   predicted_pm25_lag_3d      623 non-null    float64       
 9   predicted_pm25_nearby_avg  834 non-null    float64       
dtypes: datetime64[us](1), float64(8), object(1)
memory usage: 89.9+ KB
None
(-7.602536, 50.862218, 36.738284, 69.923179)


...

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       1149 non-null   datetime64[us]
 1   sensor_id                  1149 non-null   object        
 2   pm25                       315 non-null    float64       
 3   predicted_pm25             837 non-null    float64       
 4   days_before_forecast_day   834 non-null    float64       
 5   predicted_pm25_rolling_3d  833 non-null    float64       
 6   predicted_pm25_lag_1d      833 non-null    float64       
 7   predicted_pm25_lag_2d      728 non-null    float64       
 8   predicted_pm25_lag_3d      623 non-null    float64       
 9   predicted_pm25_nearby_avg  834 non-null    float64       
dtypes: datetime64[us](1), float64(8), object(1)
memory usage: 89.9+ KB
None
(-7.602536, 50.862218, 36.738284, 69.923179)


Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       1149 non-null   datetime64[us]
 1   sensor_id                  1149 non-null   object        
 2   pm25                       315 non-null    float64       
 3   predicted_pm25             837 non-null    float64       
 4   days_before_forecast_day   834 non-null    float64       
 5   predicted_pm25_rolling_3d  833 non-null    float64       
 6   predicted_pm25_lag_1d      833 non-null    float64       
 7   predicted_pm25_lag_2d      728 non-null    float64       
 8   predicted_pm25_lag_3d      623 non-null    float64       
 9   predicted_pm25_nearby_avg  834 non-null    float64       
dtypes: datetime64[us](1), float64(8), object(1)
memory usage: 89.9+ KB
None
(-7.602536, 50.862218, 36.738284, 69.923179)


Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…